In [ ]:
%matplotlib inline

from __future__ import division
import numpy as np
import menpo.io as mio
from menpo.image import Image
from menpo.feature import ndfeature, greyscale, centralize
from menpo.landmark import labeller, ibug_face_66
from menpo.visualize import visualize_images, print_dynamic, progress_bar_str
from menpofit.visualize import visualize_fitting_results
from alaborticcv2015.alignment import LKFitter, FilteredSSD, FilteredFourierSSD

In [ ]:
@ndfeature
def grey_igo(pixels):
    pixels = greyscale(pixels)
    pixels = igo (pixels)
    return pixels

def greyscale_average(pixels):
    return greyscale(pixels, mode='average')

# Multipie experiment: LDA Net tunning

Load training data:

In [ ]:
folder_path = '/data/PhD/alaborticcv2015/'

In [ ]:
images = []
for i in mio.import_images(folder_path + 'data/multipie/002/', verbose=True):
    i.crop_to_landmarks_proportion_inplace(0.5)
    i = i.rescale_landmarks_to_diagonal_range(200)
    labeller(i, 'PTS', ibug_face_66)
    lms = i.landmarks['ibug_face_66'].lms
    bb = lms.bounding_box()
    i.landmarks['bounding_box'] = bb
    images.append(i)

In [ ]:
template = images[7]
test_images = images

In [ ]:
visualize_images(template)

In [ ]:
visualize_images(test_images)

Define network parameters:

In [ ]:
features = greyscale_average
architecture = 1
normalize_patches = centralize
normalize_filters = None

n_filters = range(1, len(class_labels), 1)
patch_shape = [ps for ps in range(2, 11, 1)]

Define fitter parameters:

In [ ]:
diagonal = 100
scales = (1,)
max_iters = 50
noise_std = 10
n_rep = 3

Align:

In [ ]:
count = 0
results = []
errors = np.empty((len(n_filters), len(patch_shape), len(images), n_rep, max_iters+len(scales)))

# for all networks
for l, nf in enumerate(n_filters):
    for m, ps in enumerate(patch_shape):
        
        if nf < ps * ps:
        
            string = '- network {}, {}, {}: '.format(nf, ps, count)

            file_name = (folder_path
                         + 'networks/lda_net/lda_net_'
                         + features.__name__ + '_'
                         + str(1) + '_' 
                         + str(architecture) + '_'
                         + str(nf) + '_' 
                         + str(ps) + '.pkl.gz') 
            net = mio.import_pickle(file_name)

            # define kernel function
            def kernel_func(ext_shape):
                return net._compute_kernel(ext_shape=ext_shape)

            # create LK fitter
            fitter = LKFitter(template,
                              group='bounding_box',
                              features=features,
                              diagonal=diagonal,
                              scales=scales,
                              residual_cls=FilteredFourierSSD,
                              kernel_func = kernel_func,
                              eps=-1)

            # random seed
            np.random.seed(seed=0)

            # for all images
            fitter_results = []
            for j, i in enumerate(test_images):
                string2 = string + ' image {}: '.format(j)

                # for each repetition
                frs = []
                for k in range(n_rep):

                    # fit image
                    gt_s = i.landmarks['bounding_box'].lms
                    s = fitter.perturb_shape(gt_s, noise_std=noise_std)
                    fr = fitter.fit(i, s, gt_shape=gt_s, max_iters=max_iters)

                    # save fitter result
                    fr = fr.as_serializableresult()
                    frs.append(fr)

                    # compute error
                    errors[l, m, j, k] = fr.errors() 

                # save each repetition fitting result
                fitter_results.append(frs)

                # compute and print errors statistics per image
                ini_errors = errors[l, m, j, :, 0]
                ini_mean = np.mean(ini_errors)
                ini_median = np.median(ini_errors)
                ini_std = np.std(ini_errors)
                final_errors = errors[l, m, j, :, -1]
                final_mean = np.mean(final_errors)
                final_median = np.median(final_errors)
                final_std = np.std(final_errors)
                print_dynamic(string2 + 
                              '\tmean: {0:.4f} - {1:.4f} '.format(ini_mean, final_mean) +
                              ' median: {0:.4f} - {1:.4f} '.format(ini_median, final_median) +
                              ' std: {0:.4f} - {1:.4f} '.format(ini_std, final_std))

            # compute and print error statistics per network
            ini_errors = errors[l, m, :, :, 0]
            ini_mean = np.mean(ini_errors)
            ini_median = np.median(ini_errors)
            ini_std = np.std(ini_errors)
            final_errors = errors[l, m, :, :, -1]
            final_mean = np.mean(final_errors)
            final_median = np.median(final_errors)
            final_std = np.std(final_errors)
            print_dynamic(string + 
                          '\t\tmean: {0:.4f} - {1:.4f} '.format(ini_mean, final_mean) +
                          ' median: {0:.4f} - {1:.4f} '.format(ini_median, final_median) +
                          ' std: {0:.4f} - {1:.4f} \n'.format(ini_std, final_std))

            # save network fitting results
            file_name = (folder_path
                         + 'results/multipie/tunning/multipie_lda_dck_1_1_' 
                         + features.__name__ + '_'
                         + str(1) + '_' 
                         + str(architecture) + '_'
                         + str(nf) + '_' 
                         + str(ps) + '_'
                         + 'tunning.pkl.gz') 
            mio.export_pickle(fitter_results, file_name, overwrite=True)

            # save each image fitting result
            results.append(fitter_results)
            
            count += 1
    
    print '\n'

In [ ]:
# undo list nesting
fitter_results = [fr for frs in results[28] for fr in frs]

# visualize
visualize_fitting_results(fitter_results)